In [0]:
path = "drive/My Drive/Colab Notebooks/466-proj2/"
f = open(path + 'readme.txt', 'r')
print(f.read())
f.close()

In [0]:
FILE = "committee_utterances.tsv"

import pandas as pd

df = pd.read_csv(path + FILE, sep='\t')
df

,vid,fileid,cid,c_name,c_house,hid,position,pid,diarization_id,last,first,start,end,utterance_order,text
0,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,6,24,1,"Go ahead and call the hearing to order, our sp..."
1,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,24,41,2,But also to the communities and it's had other...
2,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,41,56,3,limited water resources are all challenges tha...
3,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,56,71,4,"And we've invited expert witnesses, who'll dis..."
4,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,71,88,5,Drought is certainly a critical issue for Cali...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30858,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,103,1,Hernandez,Ed,1485,1494,555,"Current vote count is 9-3, and that bill is ou..."
30859,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,2998,4,Secretary,Committee,1494,1498,556,Anderson? Mitchell? (aye) Moorlach?
30860,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,103,1,Hernandez,Ed,1504,1512,557,Current vote count is 9-2. That bill is out. I...
30861,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,2998,4,Secretary,Committee,1512,1521,558,Morrell? (no) Anderson? Mitchell? (aye) 9-3.


In [0]:
#Select a random 25% (1/4) of the content
import random

records = list(df.text)
number_selected = len(records) // 16  #TODO CHANGE THIS VALUE TO 4
selected_records = random.sample(records, number_selected) #get random sample of number_selected records without replacement

utterances = [record for record in selected_records] #get all the utterance fields only (column #15) for clustering
print("total number of points:",len(utterances))

total number of points: 1928


In [0]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
punctuation = "-!?;:\"\'.,"

def getFeatures(text):
    features = {}
    tokens = nltk.word_tokenize(text)
    for word in tokens:        
        if word not in stop_words and word not in punctuation:
            w = wordnet_lemmatizer.lemmatize(word)
            if w not in features:
                features[w] = 1
            else:
                features[w] += 1            
    return features

def computeTF(word_dict, all_words):
    tf = {}
    total_words = len(set(all_words))
    for word, count in word_dict.items():
        tf[word] = count / float(total_words)
    return tf  

import math
def computeIDF(docs):
    idf = {}
    n = len(docs)
    for d in docs:
        for word,value in d.items():
            if value > 0:
                if word not in idf:                  
                    idf[word] = 1
                else:
                    idf[word] += 1
    for word,value in idf.items():
        idf[word] = math.log(n/float(value))
    return idf

def computeTFIDF(tf_all_words, idfs):
    tfidf = {}
    for word, val in tf_all_words.items():
        tfidf[word] = val * idfs[word]
    return tfidf

def tfidf_vectorizer(feature_analyzer, data):
    # Get features
    data_dict = []
    for sample in data:
        feats = feature_analyzer(sample)
        data_dict.append(feats)
    # Get TFIDF
    matrix = []
    idf = computeIDF(data_dict)
    tfidf = computeTFIDF(computeTF())
    matrix.append()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
feats1 = getFeatures(df.text[0])
feats2 = getFeatures(df.text[2])
feats3 = getFeatures(df.text[3])
FEATS = []
FEATS.append(feats1)
FEATS.append(feats2)
FEATS.append(feats3)
SET = set(feats1).union(set(feats2)).union(set(feats3))

computeTFIDF(computeTF(set2, SET), computeIDF(FEATS))

{'AG': 0.3662040962227032,
 'This': 0.1831020481113516,
 'adaptive': 0.1831020481113516,
 'challenge': 0.1831020481113516,
 'community': 0.1831020481113516,
 'discus': 0.06757751801802739,
 'drought': 0.0,
 'hearing': 0.06757751801802739,
 'impact': 0.1831020481113516,
 'limited': 0.1831020481113516,
 'mitigate': 0.1831020481113516,
 'must': 0.1831020481113516,
 'overcome': 0.1831020481113516,
 'overview': 0.1831020481113516,
 'provide': 0.1831020481113516,
 'resource': 0.1831020481113516,
 'strategy': 0.06757751801802739,
 'today': 0.1831020481113516,
 'water': 0.06757751801802739}

In [0]:
def initialize_centroids(D, k)

def k-means(D, k, e):


In [0]:
data = df['text']
data = data[:2]

tfidf_vectorizer(getFeatures, data)

[{"'s": 1,
  'And': 2,
  'California': 1,
  'Go': 1,
  'I': 1,
  'Impacts': 1,
  'agriculture': 1,
  'ahead': 1,
  'already': 1,
  'call': 1,
  'caused': 1,
  'damage': 1,
  'drought': 2,
  'entering': 1,
  'farm': 1,
  'fifth': 1,
  'hearing': 2,
  'know': 1,
  'local': 1,
  'much': 1,
  "n't": 1,
  'order': 1,
  'ranch': 1,
  'special': 1,
  'tell': 1,
  'titled': 1,
  'year': 1},
 {"'re": 1,
  "'s": 1,
  'But': 2,
  'In': 1,
  'SIerras': 1,
  'also': 1,
  'always': 1,
  'availability': 1,
  'challenge': 1,
  'change': 1,
  'climate': 1,
  'community': 1,
  'discus': 1,
  'farming': 1,
  'impact': 1,
  'low': 1,
  'pack': 1,
  'particular': 1,
  'reliant': 1,
  'snow': 1,
  'today': 1,
  'water': 1,
  'weather': 1}]

In [0]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
data = df['text']
data = data[:2]

tf_idf_vectorizor = TfidfVectorizer(stop_words = 'english',#tokenizer = tokenize_and_stem,
                             max_features = 20000)
tf_idf = tf_idf_vectorizor.fit_transform(data)
tf_idf_norm = normalize(tf_idf)
tf_idf_array = tf_idf_norm.toarray()
print(len(data[0].split()), data[0])
print(len(data[1].split()), data[1])
print(tf_idf_array)

50 Go ahead and call the hearing to order, our special hearing titled Impacts of the drought on agriculture. And I don't have to tell you, you already know that California is entering its fifth year of drought. And it's caused so much damage to our local farms and our ranches.
47 But also to the communities and it's had other impacts that we're here to discuss today. In particular, farming has always had its challenges, because it is so reliant on the weather and availability of water. But climate change, the low snow pack in the SIerras and
[[0.19800527 0.19800527 0.         0.19800527 0.19800527 0.
  0.         0.         0.         0.19800527 0.         0.19800527
  0.39601054 0.19800527 0.         0.19800527 0.19800527 0.39601054
  0.14088238 0.19800527 0.19800527 0.         0.19800527 0.
  0.         0.19800527 0.         0.         0.         0.19800527
  0.19800527 0.19800527 0.         0.         0.         0.19800527]
 [0.         0.         0.24613641 0.         0.         0.